In [1]:
import transformers
import os
import json
import csv
from datasets import Dataset
from datasets.dataset_dict import DatasetDict
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_metric

2021-07-03 21:32:27.568506: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# context max_len : 632
# num of train data : 32640

In [3]:
with open('data/train/train.json', 'r') as f:
    train_list = json.load(f)

In [4]:
model_checkpoint = "bert-base-uncased"

In [5]:
# 開啟 CSV 檔案
with open('data/train.csv', newline='') as csvfile:
  # 讀取 CSV 檔案內容
    train_data_csv = list(csv.reader(csvfile))
print(train_data_csv[0])
train_data_csv.pop(0)

with open('data/dev.csv', newline='') as csvfile:
  # 讀取 CSV 檔案內容
    dev_data_csv = list(csv.reader(csvfile))
print(dev_data_csv[0])
dev_data_csv.pop(0)

['id', 'label', 'slot', 'context']
['id', 'label', 'slot', 'context']


['id', 'label', 'slot', 'context']

In [6]:
train_data = {'idx':[], 'label': [], 'sentence1': [], 'sentence2': []}
val_data = {'idx':[], 'label': [], 'sentence1': [], 'sentence2': []}
for i, row in enumerate(train_data_csv):
    train_data['idx'].append(row[0])
    train_data['label'].append(int(row[1]))
    train_data['sentence1'].append(row[2])
    train_data['sentence2'].append(row[3])
for i, row in enumerate(dev_data_csv):
    val_data['idx'].append(row[0])
    val_data['label'].append(int(row[1]))
    val_data['sentence1'].append(row[2])
    val_data['sentence2'].append(row[3])

dataset = DatasetDict({"train":Dataset.from_dict(train_data), "validation":Dataset.from_dict(val_data)})

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
from transformers import TrainingArguments
learning_rate = 2e-5
batch_size = 8
epochs = 3
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
#     metric_for_best_model="accuracy",
)

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    correct = 0
    for i in range(len(labels)):
        if preds[i] == labels[i]:
            correct += 1
    acc = correct / len(labels)
    return {'accuracy': acc}

In [12]:
from transformers import Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.062600,0.212809,0.959771
2,0.051100,0.210253,0.961184
3,0.029200,0.264489,0.959577


/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functio

/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/data/andy/anaconda3/envs/adlfinal/lib/python3.8/site-packages/torch/nn/parallel/_functio

TrainOutput(global_step=22182, training_loss=0.0621168375466687, metrics={'train_runtime': 13529.4962, 'train_samples_per_second': 1.64, 'total_flos': 1871710144321728.0, 'epoch': 3.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.2644892930984497,
 'eval_accuracy': 0.9595766492117586,
 'eval_runtime': 190.8914,
 'eval_samples_per_second': 189.076,
 'epoch': 3.0}

In [15]:
weights_path = "./weights"
tokenizer_path = "./tokenizer"
trainer.save_model(weights_path)
tokenizer.save_pretrained(tokenizer_path)

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [16]:
len(input_ids_list)

NameError: name 'input_ids_list' is not defined

In [ ]:
from datetime import datetime
import time
start_time = datetime.now()
start_time.strftime("%m/%d %H:%M:%S")
time.sleep(5)
end_time = datetime.now() - start_time
print('time:',end_time)

In [ ]:
torch.save(model, "./context.pkl")

In [ ]:
# test public data

import json
import torch
from transformers import (
    BertForMultipleChoice,
    BertTokenizer,
)
from dataset import contextDataset
from torch.utils.data import Dataset, DataLoader

with open('hw2_dataset/dataset/public.json', encoding="utf-8") as f:
    data_public = json.load(f)
batch_size = 32

model_checkpoint = "bert-base-chinese"  # Use chinese bert model and tokenizer
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
public_set = contextDataset(data_public, "train", tokenizer=tokenizer)
public_loader = DataLoader(public_set, batch_size = batch_size, shuffle=True, collate_fn = public_set.collate_fn)

SEED = 2021
torch.manual_seed(SEED)
torch.cuda.empty_cache()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = torch.load("./context.pkl")
model = model.to(device)

model.eval()
with torch.no_grad():
    public_acc = 0.0
    for i, batch in enumerate(public_loader):
        input_ids_list, token_type_ids_list, attention_mask_list, label_list = \
            batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(input_ids_list, token_type_ids_list, attention_mask_list, labels=label_list)
        public_acc += torch.sum(torch.argmax(outputs[1], dim=1) == label_list).item()
    public_acc /= len(public_set)